In [2]:
from keras.models import load_model
import numpy as np
from keras.utils import image_dataset_from_directory
import tensorflow as tf

In [3]:
Model = load_model('siamese_network', compile=False)

In [4]:
new = Model.layers[3]

In [5]:
import cv2 as cv
import os
import shutil

In [6]:
net = cv.dnn.readNetFromCaffe("./deploy.prototxt.txt", "./res10_300x300_ssd_iter_140000.caffemodel")

In [7]:
def cropped_face(file_path):
    image = cv.imread(file_path)
    (h, w) = image.shape[:2]
    blob = cv.dnn.blobFromImage(cv.resize(image, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    i = np.argmax(detections[0, 0, :, 2])
    confidence = np.max(detections[0, 0, :, 2])
    if confidence > 0.6:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        face = image[startY:endY,startX:endX,:]

    return face

In [7]:
def name_list(input_path):
    names=[]
    for filename in os.listdir(input_path):
        names.append(filename)
    return names

In [8]:
def make_dataset(input_path, output_path):
    names = name_list(input_path)
    for name in names:
        image = cropped_face(os.path.join(input_path, name))
        cv.imwrite(os.path.join(output_path, name), image)


In [8]:
def preprocess_file(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = image[None, :, :, :]
    return image

In [9]:
def preprocess_image(image):
    image = tf.convert_to_tensor(image)
    image = tf.image.resize(image, (224, 224))
    image = image[None, :, :, :]
    return image

In [10]:
Saptarshi = new.predict(preprocess_file("./dataset/Saptarshi.jpg"))
Shubhradeep = new.predict(preprocess_file("./dataset/Shubhradeep.jpg"))
Trijeta = new.predict(preprocess_file("./dataset/Trijeta.jpg"))

1/1 [==============================] - 0s 105ms/step


In [11]:
def who(face):
    face = preprocess_image(face)
    pred = new.predict(face, verbose=0)
    distance = []
    distance.append(np.sum(np.square(pred-Saptarshi), axis=-1))
    distance.append(np.sum(np.square(pred-Shubhradeep), axis=-1))
    distance.append(np.sum(np.square(pred-Trijeta), axis=-1))
    name = ["Saptarshi", "Shubhradeep", "Trijeta", "Unknown"]
    if np.min(distance) > 1.8:
        return name[3]
    return name[np.argmin(distance)]

In [18]:
# ACCESSING VIDEO AND PREDICTING

cap = cv.VideoCapture(0)
while True:
	ret, image = cap.read()
	if ret:
		(h, w) = image.shape[:2]
		blob = cv.dnn.blobFromImage(cv.resize(image, (300, 300)), 1.0,
			(300, 300), (104.0, 177.0, 123.0))
		net.setInput(blob)
		detections = net.forward()
		# loop over the detections
		i = np.argmax(detections[0, 0, :, 2])
		# extract the confidence (i.e., probability) associated with the
		# prediction
		confidence = np.max(detections[0, 0, :, 2])
		# filter out weak detections by ensuring the `confidence` is
		# greater than the minimum confidence
		if confidence > 0.9:
			# compute the (x, y)-coordinates of the bounding box for the
			# object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
			face = image[startY:endY,startX:endX,:]
			try:
				prediction = who(face)
				text = prediction
				y = startY - 10 if startY - 10 > 10 else startY + 10
				cv.rectangle(image, (startX, startY), (endX, endY),
								(0, 0, 255), 2)
				cv.putText(image, text, (startX, y),
							cv.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
			except:
				print("Too close to cam")
		if cv.waitKey(1) == ord('q'):
			break
		# show the output image
		cv.imshow("Output", image)
cv.destroyAllWindows()

Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
Too close to cam
